# Tutorial: Hoeffding Trees for Efficient Appointment Scheduling

## 1. Introduction: The Appointment Scheduling Challenge

Optimizing appointment schedules is a complex task. We aim to minimize patient waiting times and provider idle times, often involving computationally expensive evaluations of potential schedules. Evaluating every possible schedule modification is usually infeasible, especially if the evaluation function (like simulating patient flow) is slow.

Machine learning, particularly **stream learning**, offers a promising approach. Instead of recalculating expensive objective functions repeatedly, we can train a model to *predict* which schedule modifications are likely to be beneficial. This tutorial explores using **Hoeffding Trees**, a type of stream learning algorithm, to learn a ranking function for appointment schedules incrementally.

This document builds upon the provided Python code, explaining the concepts and how the code implements them.

## 2. Foundations of Hoeffding Trees

### 2.1. What is Stream Learning?

Stream learning deals with data that arrives sequentially and potentially infinitely. Key characteristics include:

* **Single Pass:** Models often process data only once (or a limited number of times).
* **Limited Memory:** Models cannot store the entire dataset.
* **Adaptability:** Models may need to adapt to changes in the data distribution over time (concept drift).
* **Real-time Processing:** Decisions and learning must often happen quickly.

### 2.2. The Challenge for Decision Trees in Streams

Traditional decision tree algorithms (like C4.5 or CART) typically require the entire dataset to be available to evaluate potential splits at each node. This is impractical for data streams.

### 2.3. The Hoeffding Bound: Learning from Partial Data

Hoeffding Trees leverage a statistical principle called the **Hoeffding Bound** (or Additive Chernoff Bound). It provides a guarantee about how close the *sample mean* of a random variable is to its *true mean* after observing a certain number of samples, with a given probability.

In the context of decision trees, we want to choose the best attribute to split a node on. We can estimate the "goodness" of a split (e.g., using Information Gain or Gini Impurity) based on the data samples seen *so far* at that node. The Hoeffding bound tells us how many samples (`n`) we need to observe to be confident (with probability `1 - delta`) that our observed best split attribute is the *true* best split attribute, within a certain margin.

### 2.4. How Hoeffding Trees Work

Hoeffding Trees grow incrementally as data arrives:

1.  **Initialization:** Start with a single leaf node.
2.  **Data Arrival:** When a data instance arrives, it's sorted down the tree to the appropriate leaf node based on existing split tests.
3.  **Learning at Leaves:** Sufficient statistics (e.g., counts for class labels, statistics for numerical attributes) are updated at the leaf node.
4.  **Split Attempt:** After observing a certain number of samples (the `grace_period`), the algorithm evaluates potential splits based on the accumulated statistics.
5.  **Hoeffding Bound Check:** It calculates the "goodness" (e.g., Information Gain) for the best split (`G(X_a)`) and the second-best split (`G(X_b)`). If the difference `G(X_a) - G(X_b)` is greater than the Hoeffding bound `epsilon` for the desired confidence `delta` and the number of samples `n` seen at the node, the algorithm is confident that `X_a` is truly the best split.
6.  **Splitting:** If the bound condition is met, the leaf is converted into an internal node that splits on attribute `X_a`, and new leaves are created for each branch.
7.  **No Split:** If the bound condition isn't met (or if the difference is very small, below a `tie_threshold`), the tree doesn't split yet and waits for more data.

### 2.5. Key Parameters (Refer to Code Block 2)

* `grace_period`: The minimum number of samples a leaf must observe before attempting a split. Prevents splitting based on too little data.
* `split_criterion`: The metric used to evaluate the quality of a split (e.g., `'info_gain'`, `'gini'`).
* `delta`: The confidence parameter for the Hoeffding bound (typically a small value like 1e-7). `1 - delta` is the probability that the chosen split is the correct one.
* `nominal_attributes`: A list specifying which features should be treated as categorical.
* `tie_threshold`: A threshold below which the difference between the best and second-best split is considered negligible, preventing splits in ambiguous cases.

### 2.6. Advantages

* **Incremental:** Learns instance by instance.
* **Fast:** Typically faster than traditional batch learners.
* **Memory Efficient:** Doesn't store the entire dataset.
* **Theoretical Guarantees:** Produces a tree asymptotically close to the one built by a batch learner.

### 2.7. Limitations

* **Parameter Sensitivity:** Performance can depend on parameters like `grace_period` and `delta`.
* **Suboptimal Splits:** May make less optimal splits early on if the Hoeffding bound condition is met with minimal data just past the grace period.

## 3. Applying Hoeffding Trees to Appointment Scheduling Ranking

Now, let's see how the concepts above are applied in the provided code.

### 3.1. Problem Formulation: Learning to Rank Schedule Pairs

Instead of predicting a direct outcome, we frame the problem as ranking. Given two schedules, typically a current schedule (`schedule1`) and a potential neighbor schedule derived from it (`schedule2`), we want the model to predict which one has a better (lower) objective function value.

In the code, we generate pairs where `schedule2` is a neighbor of `schedule1`. The target variable `rank` is 0 if `schedule1` is better and 1 if `schedule2` is better.

### 3.2. Feature Engineering (Refer to Code Block 4 & 6)

* **Input:** The model needs features to distinguish between schedules. The code uses a specific strategy:
    * `schedule1`: The first schedule in the pair.
    * `diff`: The *difference* between `schedule1` and `schedule2`.
* **Concatenation:** The `prepare_features` function concatenates `schedule1` and `diff` into a single feature vector.
* **Rationale:** This representation implicitly provides information about both the starting point (`schedule1`) and the modification applied (`diff`) that resulted in `schedule2`. The model learns how different modifications affect schedules based on their starting configuration.
* **Format:** The features are converted into a dictionary `{index: value}`, which is the standard input format for `river` models.

### 3.3. Target Variable Generation (Refer to Code Block 5)

* **Expensive Calculation:** The `calculate_objective_serv_time_lookup` function (assumed to be computationally expensive) is called for both schedules in each pair (`neighbor[0]` and `neighbor[1]`).
* **Comparison:** The weighted objective function values are calculated and compared.
* **Ranking:** The `rankings` list stores the index (0 or 1) of the schedule with the *minimum* objective value within each pair. This becomes the target label (`y`) for the Hoeffding Tree.

### 3.4. Data Generation Simulation (Refer to Code Block 4)

* **Initialization:** An initial schedule (`initial_x`) is created using `bailey_welch_schedule`.
* **Neighborhood Sampling:** The code simulates generating potential schedule modifications (`neighbors_selection`) based on the initial schedule and precomputed vectors (`v_star`). It then creates pairs (`neighbors_list`) where each element contains `[schedule1, schedule2, difference]`.
* **Purpose:** This simulates the kind of data stream that might be generated by an optimization algorithm like the `local_search_predict_update` function discussed previously. In a real scenario, the pairs would likely be (current\_best\_schedule, proposed\_neighbor\_schedule).

### 3.5. Model Instantiation (Refer to Code Block 2)

* The code instantiates `tree.HoeffdingTreeClassifier` with specific parameters (`grace_period=200`, `delta=1e-7`, etc.). These choices reflect a balance between reacting quickly to data and ensuring statistical significance before splitting.

### 3.6. Incremental Training & Evaluation (Refer to Code Block 6)

* **Looping:** The code iterates through the generated `neighbors_list` and `rankings`.
* **Feature Prep:** For each pair, features are prepared using `prepare_features`.
* **Prequential Evaluation:**
    * `ht_model.predict_one(x=features)`: The model predicts the rank *before* seeing the true label.
    * `metric.update(target, y_pred)`: The prediction is compared to the true `rank` (target) to update the accuracy metric. This simulates real-world performance where predictions are made on unseen data.
* **Learning:** `ht_model.learn_one(x=features, y=target)`: The model updates its structure and statistics based on the instance (features and true rank).
* **Progress:** Accuracy is printed periodically to monitor learning.

## 4. Handling Concept Drift: Adaptive Hoeffding Trees

### 4.1. What is Concept Drift?

In real-world streams, the underlying patterns can change. For appointment scheduling, this could mean:

* Changes in patient no-show rates (`q`).
* Shifts in average service times (`s`).
* Changes in the relative importance of waiting vs. idle time (`w`).

A standard Hoeffding Tree assumes a stationary distribution and might become outdated if the underlying concepts drift significantly.

### 4.2. Hoeffding Adaptive Trees (HAT / ARF) (Refer to Code Block 2 & 7)

* **Mechanism:** Adaptive variants like `HoeffdingAdaptiveTreeClassifier` (HAT) or ensemble methods like Adaptive Random Forests (ARF, which often uses HAT as a base learner) incorporate **drift detectors**.
* **Drift Detection:** Algorithms like ADWIN (`drift.ADWIN`) monitor the model's performance (e.g., error rate). If a statistically significant change is detected, it signals a drift.
* **Adaptation:** When drift is detected, the model adapts. HAT, for instance, might start growing a "background" tree. If the background tree becomes more accurate than the original, it replaces the original tree at the affected node. ARF manages an ensemble of trees, replacing drifting trees with new ones.
* **Instantiation:** The code shows instantiating `HoeffdingAdaptiveTreeClassifier` with `drift_detector=drift.ADWIN(...)`. Note that the drift detector itself has parameters (like its own `delta`) controlling its sensitivity.
* **Training:** The training loop (Code Block 7) is identical to the standard Hoeffding Tree loop, as the drift detection and adaptation happen internally within the `learn_one` method of the adaptive model.

## 5. Integration with Optimization (Conceptual)

The trained Hoeffding Tree (standard or adaptive) serves as a fast proxy for the expensive objective function. In the `local_search_predict_update` function (from the Canvas), instead of always calling `calculate_objective_serv_time_lookup`, we could:

1.  Generate a `feature_pair` (e.g., `x_star + neighbor`).
2.  Prepare features: `features = prepare_features(x_star, neighbor - x_star)` (assuming `prepare_features` is adapted).
3.  Predict ranking: `predicted_rank = ht_model.predict_one(features)`.
4.  Use the prediction: If `predicted_rank` suggests the neighbor is better (e.g., `predicted_rank == 1` if 1 means neighbor is better), prioritize this neighbor for the *expensive* check, or even tentatively accept it if confidence is high. If it predicts the neighbor is worse, potentially skip the expensive check entirely.

This hybrid approach aims to drastically reduce the number of expensive objective calls, speeding up the search process. The Hoeffding Tree continuously learns and adapts from the *results* of the expensive checks that *are* performed (especially when mispredictions occur).

## 6. Conclusion

Hoeffding Trees provide a powerful framework for building decision trees on streaming data, making them suitable for applications like learning heuristics within complex optimization processes like appointment scheduling. By learning to rank potential schedule modifications incrementally, they can help guide search algorithms towards better solutions more efficiently than relying solely on expensive evaluations. Adaptive variants further enhance their robustness by handling changes in the underlying scheduling environment over time.

**Next Steps:**

* Experiment with different Hoeffding Tree parameters (`grace_period`, `delta`, `split_criterion`).
* Explore alternative feature representations for the schedule pairs.
* Compare the performance of standard vs. adaptive trees.
* Implement the integration of the trained tree into the `local_search_predict_update` function.
* Investigate other stream learning models available in libraries like `river`.


In [ ]:
# Import necessary river components
import time
import math
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import pickle
import random
from scipy.optimize import minimize
from river import tree, metrics, evaluate, stream, drift

In [ ]:
# Instantiate the Hoeffding Tree Classifier
# These parameters are illustrative; optimal values depend on the specific data stream
ht_model = tree.HoeffdingTreeClassifier(
    grace_period=200,           # Number of samples to observe before considering a split
    split_criterion='info_gain',# Criterion for choosing splits (Information Gain)
    delta=1e-7,                 # Hoeffding bound confidence parameter (1 - delta)
    nominal_attributes=None     # Specify if any features are nominal (categorical)
    # Other parameters like split_confidence, tie_threshold can also be set
)

print("Hoeffding Tree Classifier instantiated.")

arf_model = tree.HoeffdingAdaptiveTreeClassifierhat_model = tree.HoeffdingAdaptiveTreeClassifier(
    grace_period=200,         # Hoeffding Tree: Wait for 100 samples before first split check
    delta=1e-7,               # Hoeffding Tree: Confidence level for Hoeffding bound
    drift_detector=drift.ADWIN(delta=0.001), # Use ADWIN detector with its own delta (sensitivity)
    switch_significance=0.05, # Significance level (beta) to switch trees after drift
    # Other relevant HT parameters (e.g., split_criterion, max_depth) can be set here
    seed=42                   # For reproducibility
)


print("Adaptive Random Forest Classifier instantiated.")

In [ ]:
from functions import compute_convolutions, bailey_welch_schedule

N = 22 # Number of patients
T = 20 # Number of intervals
d = 5 # Length of each interval
max_s = 20 # Maximum service time
q = 0.20 # Probability of a scheduled patient not showing up
w = 0.1 # Weight for the waiting time in objective function
l = 10
num_schedules = 60000 # Number of schedules to sample

# Create service time distribution
def generate_weighted_list(max_s, l, i):
    """
    Generates a service time probability distribution using optimization.

    This function creates a discrete probability distribution over T possible
    service times (from 1 to T). It uses optimization (SLSQP) to find a
    distribution whose weighted average service time is as close as possible
    to a target value 'l', subject to the constraint that the probabilities
    sum to 1 and each probability is between 0 and 1.

    After finding the distribution, it sorts the probabilities: the first 'i'
    probabilities (corresponding to service times 1 to i) are sorted in
    ascending order, and the remaining probabilities (service times i+1 to T)
    are sorted in descending order.

    Note:
        - This function relies on a globally defined integer 'T', representing
          the maximum service time considered (or number of probability bins).
        - The parameter 'max_s' is accepted but not used directly within this
          function's optimization or sorting logic as shown. It might be
          related to how 'T' is determined externally.
        - Requires NumPy and SciPy libraries (specifically scipy.optimize.minimize).

    Args:
        max_s (any): Maximum service time parameter (currently unused in the
                     provided function body's core logic).
        l (float): The target weighted average service time for the distribution.
        i (int): The index determining the sorting split point. Probabilities
                 for service times 1 to 'i' are sorted ascendingly, and
                 probabilities for service times 'i+1' to 'T' are sorted
                 descendingly. Must be between 1 and T-1 for meaningful sorting.

    Returns:
        numpy.ndarray: An array of size T+1. The first element (index 0) is 0.
                       Elements from index 1 to T represent the calculated
                       and sorted probability distribution, summing to 1.
                       Returns None if optimization fails.
    """
    # Initialize an array of T+1 values, starting with zero
    # Index 0 is unused for probability, indices 1 to T hold the distribution
    values = np.zeros(T + 1)

    # --- Inner helper function for optimization ---
    def objective(x):
        """Objective function: Squared difference between weighted average and target l."""
        # Calculate weighted average: sum(index * probability) / sum(probability)
        # Since sum(probability) is constrained to 1, it simplifies.
        weighted_avg = np.dot(np.arange(1, T + 1), x) # Corresponds to sum(k * P(ServiceTime=k))
        return (weighted_avg - l) ** 2

    # --- Constraints for optimization ---
    # Constraint 1: The sum of the probabilities (x[0] to x[T-1]) must be 1
    constraints = ({
        'type': 'eq',
        'fun': lambda x: np.sum(x) - 1
    })

    # Bounds: Each probability value x[k] must be between 0 and 1
    # Creates a list of T tuples, e.g., [(0, 1), (0, 1), ..., (0, 1)]
    bounds = [(0, 1)] * T

    # Initial guess: Use Dirichlet distribution to get a random distribution that sums to 1
    # Provides a starting point for the optimizer. np.ones(T) gives equal weights initially.
    initial_guess = np.random.dirichlet(np.ones(T))

    # --- Perform Optimization ---
    # Minimize the objective function subject to the sum and bounds constraints
    # using the Sequential Least Squares Programming (SLSQP) method.
    result = minimize(objective, initial_guess, method='SLSQP', bounds=bounds, constraints=constraints)

    # Check if optimization was successful
    if not result.success:
        print(f"Warning: Optimization failed! Message: {result.message}")
        # Handle failure case, e.g., return None or raise an error
        return None # Or potentially return a default distribution

    # Assign the optimized probabilities (result.x) to the correct slice of the values array
    # result.x contains the T probabilities for service times 1 to T.
    values[1:] = result.x

    # --- Reorder the values based on the index 'i' ---
    # Ensure 'i' is within a valid range for slicing and sorting
    if not (0 < i < T):
       print(f"Warning: Index 'i' ({i}) is outside the valid range (1 to {T-1}). Sorting might be trivial.")
       # Adjust i or handle as an error depending on requirements
       i = max(1, min(i, T - 1)) # Clamp i to a safe range for demonstration

    # Sort the first 'i' probabilities (indices 1 to i) in ascending order
    first_part = np.sort(values[1:i+1])
    # Sort the remaining 'T-i' probabilities (indices i+1 to T) in descending order
    second_part = np.sort(values[i+1:])[::-1] # [::-1] reverses the sorted array

    # Combine the sorted parts back into the 'values' array
    values[1:i+1] = first_part
    values[i+1:] = second_part

    # Return the final array with the sorted probability distribution
    return values

i = 5  # First 5 highest values in ascending order, rest in descending order
s = generate_weighted_list(max_s, l, i)
print(s)
print("Sum:", np.sum(s[1:]))  # This should be 1
print("Weighted service time:", np.dot(np.arange(len(s)), s))  # This should be close to l
initial_x = bailey_welch_schedule(T, d, N, s)
print(f"Initial schedule: {initial_x}")
convolutions = compute_convolutions(s, N, q)

In [ ]:
from functions import get_v_star, get_neighborhood

def sample_neighbors_list(x: list[int], v_star: np.ndarray, all = True) -> (list[int], list[int]):
    """
    Create a set of pairs of schedules that are from the same neighborhood.
    
    Parameters:
      x (list[int]): A list of integers with |s| = T and sum N.
      v_star (np.ndarray): Precomputed vectors V* of length T.
      
    Returns:
      tuple(list[int], list[int]): A pair of schedules.
    """
    T = len(x)

    # Precompute binomial coefficients (weights for random.choices)
    binom_coeff = [math.comb(T, i) for i in range(1, T)]

    # Choose a random value of i with the corresponding probability
    i = random.choices(range(1, T), weights=binom_coeff)[0]

    # Instead of generating the full list of combinations, sample one directly
    j = random.sample(range(T), i)
    
    x_p = x.copy()
    for k in j:
        x_temp = np.array(x_p) + v_star[k]
        x_temp = x_temp.astype(int)
        if np.all(x_temp >= 0):
            x_p = x_temp.astype(int).tolist()
    if all:
        diff = [int(x - y) for x, y in zip(x, x_p)]
        return x, x_p, diff
    else:    
        return x_p

start = time.time()
v_star = get_v_star(T)
# Sample a set of schedules from the neighborhood of the initial schedule
neighbors_selection = [sample_neighbors_list(initial_x, v_star, all = False) for i in range(num_schedules)] # This can be done in parallel to improve speed
print(len(neighbors_selection))
end = time.time()
# For the sampled schedules, create the neighbors
neighbors_list = [sample_neighbors_list(schedule, v_star) for schedule in neighbors_selection]
# Randomly switch the order of the neighbors
# neighbors_list = [neighbor if random.random() < 0.5 else neighbor[::-1] for neighbor in neighbors_list]
end = time.time()
h = random.choices(range(num_schedules), k=7)
print(f"Sampled schedules: {h}")
for i in h:
    original_schedule = neighbors_list[i][0]
    neighbor_schedule = neighbors_list[i][1]
    difference =neighbors_list[i][2]
    print(f"Neighbors\n{original_schedule}\n{neighbor_schedule}\n{difference}")
training_set_feat_time = end - start
print(f"\nProcessing time: {training_set_feat_time} seconds\n")

In [ ]:
from functions import calculate_objective_serv_time_lookup

objectives_schedule_1 = [
    w * result[0] + (1 - w) * result[1]
    for neighbor in neighbors_list
    for result in [calculate_objective_serv_time_lookup(neighbor[0], d, convolutions)]
]
start = time.time()
objectives_schedule_2 = [
    w * result[0] + (1 - w) * result[1]
    for neighbor in neighbors_list
    for result in [calculate_objective_serv_time_lookup(neighbor[1], d, convolutions)]
]
end = time.time()
training_set_lab_time = end - start
objectives = [[obj, objectives_schedule_2[i]] for i, obj in enumerate(objectives_schedule_1)]
rankings = np.argmin(objectives, axis=1).tolist()
for i in range(5):
    print(f"Objectives: {objectives[i]}, Ranking: {rankings[i]}")

print(f"\nProcessing time: {training_set_lab_time} seconds\n")

# Step 1: Flatten the objectives into a 1D array
flattened_data = [value for sublist in objectives for value in sublist]

# Step 2: Find the index of the minimum value
min_index = np.argmin(flattened_data)

# Step 3: Convert that index back to the original 2D structure
row_index = min_index // 2  # Assuming each inner list has 2 values
col_index = min_index % 2

print(f"The minimum objective value is at index [{row_index}][{col_index}].\nThis is schedule: {neighbors_list[row_index][col_index]} with objective value {objectives[row_index][col_index]}.")

print(f"\nAverage ranking: {np.mean(rankings)}\n")

In [ ]:
def prepare_features(schedule1, diff):
    """
    Concatenates two schedule vectors and converts them into a
    dictionary format suitable for river models.
    """
    # Ensure schedules are numpy arrays for easier concatenation if needed
    s1 = np.asarray(schedule1)
    diff = np.asarray(diff)
    
    # Concatenate the two schedule vectors
    concatenated_schedule = np.concatenate((s1, diff))
    
    # Create feature dictionary using indices as keys
    # Example: {0: val_s1_0, 1: val_s1_1,..., N: val_s2_0,...}
    features = {i: float(val) for i, val in enumerate(concatenated_schedule)}
    return features

print("\nStarting incremental training simulation...")
# Use river's stream utilities for evaluation (optional but good practice)
metric = metrics.Accuracy()

for i, (sched_pair, rank) in enumerate(zip(neighbors_list, rankings)):
    schedule1 = sched_pair[0]
    diff = sched_pair[2]
    features = prepare_features(schedule1, diff)
    target = rank

    # Optional: Make a prediction before learning (for prequential evaluation)
    y_pred = ht_model.predict_one(x=features)
    if y_pred is not None: # Model might not be ready to predict initially
        metric.update(target, y_pred)

    # Learn from the current instance (pair + rank)
    ht_model.learn_one(x=features, y=target)

    if (i + 1) % 1000 == 0: # Print progress for every 100th pair in this small example
        print(f"Processed pair {i+1}. Current Accuracy: {metric.get():.4f}") # If using prequential evaluation
        print(f"Processed pair {i+1}. Model updated.")

print("Incremental training simulation finished.")
print(f"Final Accuracy (Prequential): {metric.get():.4f}") # If using prequential evaluation

In [ ]:
metric_arf = metrics.Accuracy()

for i, (sched_pair, rank) in enumerate(zip(neighbors_list, rankings)):
    schedule1 = sched_pair[0]
    diff = sched_pair[2]
    features = prepare_features(schedule1, diff)
    target = rank

    # Optional: Make a prediction before learning (for prequential evaluation)
    y_pred = arf_model.predict_one(x=features)
    if y_pred is not None: # Model might not be ready to predict initially
        metric_arf.update(target, y_pred)

    # Learn from the current instance (pair + rank)
    arf_model.learn_one(x=features, y=target)

    if (i + 1) % 1000 == 0: # Print progress for every 100th pair in this small example
        print(f"Processed pair {i+1}. Current Accuracy: {metric_arf.get():.4f}") # If using prequential evaluation
        print(f"Processed pair {i+1}. Model updated.")

print("Incremental training simulation finished.")
print(f"Final Accuracy (Prequential): {metric_arf.get():.4f}") # If using prequential evaluation